In [51]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from scipy.stats import spearmanr

In [52]:
    
#path = "./Data/CAGE-train/"
path = '/Users/nidhiagrawal/Desktop/Assignments/2nd Semester/ML in Genomics/Project 1/Data/CAGE-train/'

X1_train = pd.read_csv(path + "X1_train_info.tsv", sep= '\t')
X1_train_y = pd.read_csv(path + "X1_train_y.tsv", sep= '\t')

X1_val = pd.read_csv(path + "X1_val_info.tsv", sep= '\t')
X1_val_y = pd.read_csv(path + "X1_val_y.tsv", sep= '\t')

X2_train = pd.read_csv(path + "X2_train_info.tsv", sep= '\t')
X2_train_y = pd.read_csv(path + "X2_train_y.tsv", sep= '\t')

X2_val = pd.read_csv(path + "X2_val_info.tsv", sep= '\t')
X2_val_y = pd.read_csv(path + "X2_val_y.tsv", sep= '\t')

X3_test = pd.read_csv(path + "X3_test_info.tsv", sep= '\t')



In [53]:

# get all .bed datasets in a dictionary
df_DNase_histones = {'DNase':[], 'H3K4me1':[], 'H3K4me3':[], 'H3K27ac':[], 'H3K27me3':[], 'H3K36me3':[]}
for name, df_cell_list in df_DNase_histones.items():
    path = './Data/'+name+'-bed/'
    for i in range(1,4):
        X = pd.read_csv(path + 'X' + str(i) + ".bed", sep= '\t')
        X.columns = ['chr', 'start','end', 'name','display_score','strand','signal_value',
                                  'pValue','qValue','peak']
        df_cell_list.append(X)


In [54]:
    

def window_peaks_data(window_data_df, df_DNase_histones, cell_line_num):
    
    maxSignalVal_list = []
    
    for name, df_list in df_DNase_histones.items():
    
        df = df_list[cell_line_num-1] # cell line's DNase or histone data
        
        if name == 'H3K36me3':
                peak_data = df.loc[(df['chr'] == window_data_df['chr']) & 
                                      (df['start'] >= window_data_df['gene_start']) & 
                                      (df['end'] <= window_data_df['gene_end'])]
        else:
            peak_data = df.loc[(df['chr'] == window_data_df['chr']) & 
                                  (df['start'] >= window_data_df['window_start']) & 
                                  (df['end'] <= window_data_df['window_end'])]
        if len(peak_data) == 0:
            maxSignalVal = 1e-4
        else:
            # get row from .bed file with max signal value
            peak_data_max = peak_data.loc[peak_data['signal_value'].idxmax()]
            maxSignalVal = peak_data_max['signal_value']

        maxSignalVal_list.append(maxSignalVal)

    return maxSignalVal_list

    
def getMaxSignalData(X_train, df_DNase_histones, window_size, cell_line_num):

    window_data_df = pd.DataFrame(X_train[['gene_name', 'chr', 'gene_start', 'gene_end', 'TSS_start']])
    window_data_df['window_start'] = X_train['TSS_start']-window_size
    window_data_df['window_end'] = X_train['TSS_start']+window_size


    maxSignalVal_list = window_data_df.apply(window_peaks_data, args=[df_DNase_histones, cell_line_num], axis=1)

    # process maxSignalVal_list to see data in dataframe
    maxSignalVal_df = pd.DataFrame(maxSignalVal_list)
    maxSignalVal_df = maxSignalVal_df[0].apply(pd.Series)
    maxSignalVal_df.columns = ['DNase','H3K4me1','H3K4me3','H3K27ac','H3K27me3','H3K36me3']
    
    return maxSignalVal_df


In [55]:

# Generate datasets to be used for training model

# window_size = Num of kb region to scan around TSS

# X1 Train dataset
maxSignalVal_X1_train = getMaxSignalData(X1_train, df_DNase_histones, window_size=3500, cell_line_num=1)
x1_train = maxSignalVal_X1_train.values
y1_train = X1_train_y['gex'].values
maxSignalVal_X1_train.insert(0, 'gene_name', X1_train['gene_name'])
maxSignalVal_X1_train.insert(7, 'gene_exp', X1_train_y['gex'])
maxSignalVal_X1_train.to_csv('X1_train_window3500.csv')

# X1 Validation dataset
maxSignalVal_X1_val = getMaxSignalData(X1_val, df_DNase_histones, window_size=3500, cell_line_num=1)
x1_val = maxSignalVal_X1_val.values
y1_val = X1_val_y['gex'].values
maxSignalVal_X1_val.insert(0, 'gene_name', X1_val['gene_name'])
maxSignalVal_X1_val.insert(7, 'gene_exp', X1_val_y['gex'])
maxSignalVal_X1_val.to_csv('X1_val_window3500.csv')

# X2 Train dataset
maxSignalVal_X2_train = getMaxSignalData(X2_train, df_DNase_histones, window_size=3500, cell_line_num=2)
x2_train = maxSignalVal_X2_train.values
y2_train = X2_train_y['gex'].values
maxSignalVal_X2_train.insert(0, 'gene_name', X2_train['gene_name'])
maxSignalVal_X2_train.insert(7, 'gene_exp', X2_train_y['gex'])
maxSignalVal_X2_train.to_csv('X2_train_window3500.csv')

# X2 Validation dataset
maxSignalVal_X2_val = getMaxSignalData(X2_val, df_DNase_histones, window_size=3500, cell_line_num=2)
x2_val = maxSignalVal_X2_val.values
y2_val = X2_val_y['gex'].values
maxSignalVal_X2_val.insert(0, 'gene_name', X2_val['gene_name'])
maxSignalVal_X2_val.insert(7, 'gene_exp', X2_val_y['gex'])
maxSignalVal_X2_val.to_csv('X2_val_window3500.csv')

# X3 Test dataset
x3_test = getMaxSignalData(X3_test, df_DNase_histones, window_size=3500, cell_line_num=3)
x3_test.to_csv('X3_test_window3500.csv')



# Linear Regression

In [59]:
y1_train = X1_train_y['gex'].values
y2_train = X2_train_y['gex'].values

# X1 cell line
reg = LinearRegression().fit(x1_train, y1_train)
y1_pred = reg.predict(x1_val)
print('X1 Val: ', spearmanr(y1_val, y1_pred))

# X2 cell line
reg = LinearRegression().fit(x2_train, y2_train)
y2_pred = reg.predict(x2_val)
print('X2 Val: ', spearmanr(y2_val, y2_pred))


X1 Val:  SpearmanrResult(correlation=0.6793704899984753, pvalue=2.1675905276751324e-267)
X2 Val:  SpearmanrResult(correlation=0.6189132103533664, pvalue=4.430419814971816e-209)


In [75]:

# X1 + X2 cell lines
x_train = np.concatenate((x1_train, x2_train), axis=0)
y_train = np.concatenate((y1_train, y2_train), axis=0)
x_val = np.concatenate((x1_val, x2_val), axis=0)
y_val = np.concatenate((y1_val, y2_val), axis=0)

reg = LinearRegression().fit(x_train, y_train)
y_pred = reg.predict(x_val)
print('X Val: ', spearmanr(y_val, y_pred))


X Val:  SpearmanrResult(correlation=0.6561773202356411, pvalue=0.0)


### Notes:

1. H3K36me3 is present in gene body and may not be near TSS - from gene C11orf58
2. H3K27me3 is a repressor
3. Others are enhancers 
4. Not using H3K9me3 as it's not normalized
5. Density plots in tutorial +- 5 kb so tried that
6. Tried diff window sizes, 3500 worked well
7. Tried Poly Linear regression but it didn't work well


# Polynomial Linear regression

In [74]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree = 2)
X_train_poly = poly.fit_transform(x_train)

poly.fit(X_train_poly, y_train)
lin2 = LinearRegression()
lin2.fit(X_train_poly, y_train)

X_val_poly = poly.fit_transform(x_val)
y_pred = lin2.predict(X_val_poly)

spearmanr(y_val, y_pred)


SpearmanrResult(correlation=0.6386043235746472, pvalue=0.0)